# [George McNinch](http://gmcninch.math.tufts.edu) Math 87 - Spring 2026

# Week 5
# Linear programs and "binding/active" constraints 

Let's unpack some more of the output of the `scipy.optimize.linprog` solver.

In [16]:
import numpy as np
from scipy.optimize import linprog

# maximize x + y
c = np.array([1, 1])

# Subject to:
# x + y <= 4      (constraint 0) 
# x <= 2          (constraint 1) 
# y <= 2          (constraint 2) 

A_ub = np.array([[1, 1],
                 [1, 0],
                 [0, 1]])

b_ub = np.array([4, 2, 2])

result = linprog((-1)*c, A_ub=A_ub, b_ub=b_ub)


Let's look at the `result` just computed.

In [17]:
result

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -4.0
              x: [ 2.000e+00  2.000e+00]
            nit: 0
          lower:  residual: [ 2.000e+00  2.000e+00]
                 marginals: [ 0.000e+00  0.000e+00]
          upper:  residual: [       inf        inf]
                 marginals: [ 0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00  0.000e+00]
                 marginals: [-0.000e+00 -1.000e+00 -1.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

The linear program had 2 variables (2 is the length of the objective function `c`), and
3 inequality constraints (3 is the number of rows of the matrix `A_ub`).

So we know that the dual linear program has 3 variables (labelled by the rows of the inequality constraint matrix).

We've seen that `fun` represents the optimal value attained by the objective function.

And `x` represents a feasible point at which the optimal value is attained. In particular, `x` is a "vector" (really, an `np.array`)
whose length is equal to the number of variables of the linear program -- in this case, `result.x` has length 2.

In [5]:
result.x.shape

(2,)

We haven't discussed it yet, but the dual `LP` of an LP has a variable for each equality constraint and a variable for each inequality constraint. 
The information in `result.eqlin` tells us about the equality constraint variables (in this case, there aren't any) and the information in
`result.ineqlin` tells us about the inequality constraint variables.

More precisely, it gives us the `residual` or `slack` information, namely the difference `b_ub - A_ub @ x`.


In [9]:
result.ineqlin.residual, b_ub - A_ub @ result.x

(array([0., 0., 0.]), array([0., 0., 0.]))

while `result.ineqlin.marginal` tells us the `dual prices` -- i.e. an optimal solution for dual linear program.

In [11]:
result.ineqlin.marginals

array([-0., -1., -1.])

In [25]:
# compare with the dual LP
dual_result = linprog(b_ub,(-1)*A_ub.transpose(),(-1)*c)
dual_result.x

array([0., 1., 1.])

Note that all three constraints have zero slack -- the maximum value is met by the given optimal solution.

On the other hand, not all of the constraints are *active* because the dual price of the first variable is zero.
What this means is that changing the first inequality from `X  + Y <= 4` to `X + Y <= 5` wouldn't by itself improve the optimal result. 

Relaxing it wouldn't help because we're already limited by the other constraints. It's a redundant constraint—accidentally tight at the optimum but not actually constraining the solution.

On the other hand, let's look at what happens when we modify one of the other constraints.

In [28]:

# Subject to:
# x + y <= 4      (constraint 0) 
# x <= 3          (constraint 1) 
# y <= 2          (constraint 2) 

b_ub_updated = b_ub + np.array([0,1,0])

result_updated = linprog((-1)*c, A_ub=A_ub, b_ub=b_ub_updated)
result_updated

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -4.0
              x: [ 2.000e+00  2.000e+00]
            nit: 0
          lower:  residual: [ 2.000e+00  2.000e+00]
                 marginals: [ 0.000e+00  0.000e+00]
          upper:  residual: [       inf        inf]
                 marginals: [ 0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  1.000e+00  0.000e+00]
                 marginals: [-1.000e+00 -0.000e+00 -0.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

After the update, it is the first constraint that becomes binding!
